In [1]:
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [2]:
df = pd.read_csv('../Data/Raw/steam_store_dlc.csv', keep_default_na=True)
df.head()

,Unnamed: 0,type,name,steam_appid,required_age,is_free,developers,publishers,price_overview,platforms,categories,release_date,controller_support,dlc,metacritic
0,0,music,The Dota 2 Official Soundtrack,1241930,0.0,False,['Valve'],['Valve'],"{'currency': 'SGD', 'initial': 1000, 'final': 200, 'discount_percent': 80, 'initial_formatted': 'S$10.00', 'final_formatted': 'S$2.00'}","{'windows': True, 'mac': True, 'linux': True}","[{'id': 50, 'description': 'Additional High-Quality Audio'}]","{'coming_soon': False, 'date': '10 Feb, 2020'}",NaN,NaN,NaN
1,1,music,The Dota 2 Remixes EP,652720,0.0,True,['Valve'],['Valve'],NaN,"{'windows': True, 'mac': True, 'linux': True}","[{'id': 50, 'description': 'Additional High-Quality Audio'}]","{'coming_soon': False, 'date': '10 Feb, 2020'}",NaN,NaN,NaN
2,2,music,Fight Songs: The Music Of Team Fortress 2,629330,0.0,False,['Valve'],['Valve'],"{'currency': 'SGD', 'initial': 1300, 'final': 260, 'discount_percent': 80, 'initial_formatted': 'S$13.00', 'final_formatted': 'S$2.60'}","{'windows': True, 'mac': True, 'linux': True}","[{'id': 50, 'description': 'Additional High-Quality Audio'}]","{'coming_soon': False, 'date': '4 May, 2017'}",NaN,NaN,NaN
3,3,dlc,Grand Theft Auto V - Criminal Enterprise Starter Pack,771300,0.0,False,['Rockstar North'],['Rockstar Games'],"{'currency': 'SGD', 'initial': 1390, 'final': 1390, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': 'S$13.90'}","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'id': 1, 'description': 'Multi-player'}, {'id': 21, 'description': 'Downloadable Content'}, {'id': 22, 'description': 'Steam Achievements'}, {'id': 28,...","{'coming_soon': False, 'date': '14 Dec, 2017'}",full,NaN,NaN
4,4,dlc,Unturned - Permanent Gold Upgrade,306460,0.0,False,['Smartly Dressed Games'],['Smartly Dressed Games'],"{'currency': 'SGD', 'initial': 525, 'final': 393, 'discount_percent': 25, 'initial_formatted': 'S$5.25', 'final_formatted': 'S$3.93'}","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'id': 1, 'description': 'Multi-player'}, {'id': 9, 'description': 'Co-op'}, {'id': 21, 'description': 'Downloadable Content'}, {'id': 22, 'description'...","{'coming_soon': False, 'date': '7 Jul, 2014'}",NaN,NaN,NaN


In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
print(df.shape)
df.isnull().sum()

(40622, 14)


type                     27
name                     27
steam_appid               0
required_age             27
is_free                  27
developers              179
publishers               27
price_overview         3702
platforms                27
categories             2605
release_date             27
controller_support    30128
dlc                   40620
metacritic            40431
dtype: int64

## Clean Data

### Type

In [5]:
df['type'].unique()

array(['music', 'dlc', nan, 'game', 'movie'], dtype=object)

In [6]:
# Keep only rows with dlc type
df = df[df['type']=='dlc']
df.shape

(36370, 14)

In [7]:
df.isnull().sum()

type                      0
name                      0
steam_appid               0
required_age              0
is_free                   0
developers              148
publishers                0
price_overview         3297
platforms                 0
categories                0
release_date              0
controller_support    26082
dlc                   36369
metacritic            36196
dtype: int64

### Developers

In [8]:
# Replace NaN with empty list
df['developers'] = df['developers'].fillna('[]')
df['developers'].head()

3            ['Rockstar North']
4     ['Smartly Dressed Games']
9         ['Facepunch Studios']
10        ['Facepunch Studios']
11        ['Facepunch Studios']
Name: developers, dtype: object

### Categories

In [9]:
dic = {}
for arr in df['categories'].values:
    if type(arr) == str:
        arr = eval(arr)
        for data in arr:
            if data["id"] not in dic.keys():
                dic[data["id"]] = set([data['description']])
            else:
                dic[data["id"]].add(data['description'])

dic

{2: {'Single-player'},
 1: {'Multi-player'},
 21: {'Downloadable Content'},
 22: {'Steam Achievements'},
 28: {'Full controller support'},
 9: {'Co-op'},
 29: {'Steam Trading Cards'},
 30: {'Steam Workshop'},
 23: {'Steam Cloud'},
 15: {'Stats'},
 17: {'Includes level editor'},
 20: {'MMO'},
 49: {'PvP'},
 36: {'Online PvP'},
 38: {'Online Co-op'},
 27: {'Cross-Platform Multiplayer'},
 35: {'In-App Purchases'},
 8: {'Valve Anti-Cheat enabled'},
 42: {'Remote Play on Tablet'},
 41: {'Remote Play on Phone'},
 18: {'Partial Controller Support'},
 24: {'Shared/Split Screen'},
 37: {'Shared/Split Screen PvP'},
 39: {'Shared/Split Screen Co-op'},
 43: {'Remote Play on TV'},
 44: {'Remote Play Together'},
 51: {'Steam Workshop'},
 54: {'VR Only'},
 31: {'VR Support'},
 40: {'SteamVR Collectibles'},
 47: {'LAN PvP'},
 48: {'LAN Co-op'},
 13: {'Captions available'},
 25: {'Steam Leaderboards'},
 32: {'Steam Turn Notifications'},
 52: {'Tracked Controller Support'},
 16: {'Includes Source SDK'},

In [10]:
mapping = {1: 'Multi-player',
 9: 'Co-op',
 29: 'Steam Trading Cards',
 30: 'Steam Workshop',
 40: 'SteamVR Collectibles',
 35: 'In-App Purchases',
 8: 'Valve Anti-Cheat enabled',
 22: 'Steam Achievements',
 28: 'Full controller support',
 15: 'Stats',
 41: 'Remote Play on Phone',
 42: 'Remote Play on Tablet',
 43: 'Remote Play on TV',
 49: 'PvP', 
 36: 'Online PvP',
 38: 'Online Co-op', 
 20: 'MMO',
 27: 'Cross-Platform Multiplayer', 
 13: 'Captions available', 
 18: 'Partial Controller Support',
 17: 'Includes level editor',
 14: 'Commentary available',
 2: 'Single-player',
 47: 'LAN PvP',
 48: 'LAN Co-op',
 23: 'Steam Cloud',
 16: 'Includes Source SDK',
 44: 'Remote Play Together',
 37: 'Shared/Split Screen PvP',
 39: 'Shared/Split Screen Co-op',
 24: 'Shared/Split Screen', 
 53: 'VR Supported',
 51: 'Steam Workshop',
 52: 'Tracked Controller Support',
 25: 'Steam Leaderboards',
 31: 'VR Support',
 32: 'Steam Turn Notifications',
 54: 'Solo RV',
 19: 'Mods',
 6: 'Mods (require HL2)',
 10: 'Game demo',
 21: 'Downloadable Content',
 50: 'Additional High-Quality Audio',
 33: 'Native Steam Controller Support',
 0: 'No Category'}

In [11]:
# Replace categories with their ID
df['categories'] = df['categories'].apply(lambda x: [dic['id'] for dic in (eval(x) if type((x)) == str else [{"id":"0"}])])

# Map the genre values to genre names using the mapping dictionary
df['categories'] = df['categories'].apply(lambda x: [mapping.get(i) for i in x])

df.head(1)

,type,name,steam_appid,required_age,is_free,developers,publishers,price_overview,platforms,categories,release_date,controller_support,dlc,metacritic
3,dlc,Grand Theft Auto V - Criminal Enterprise Starter Pack,771300,0.0,False,['Rockstar North'],['Rockstar Games'],"{'currency': 'SGD', 'initial': 1390, 'final': 1390, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': 'S$13.90'}","{'windows': True, 'mac': False, 'linux': False}","[Single-player, Multi-player, Downloadable Content, Steam Achievements, Full controller support]","{'coming_soon': False, 'date': '14 Dec, 2017'}",full,NaN,NaN


### Release Date

In [12]:
df['release_date'] = df['release_date'].apply(lambda x: eval(x) if type(x) == str else x)

In [13]:
# Number of unreleased games
temp = df[~df['release_date'].isnull()]
len(temp[temp['release_date'].apply(lambda x: x['coming_soon'])])

170

In [14]:
temp[temp['release_date'].apply(lambda x: x['coming_soon'])]['release_date']

424      {'coming_soon': True, 'date': 'To be announced'}
426          {'coming_soon': True, 'date': 'Coming soon'}
822         {'coming_soon': True, 'date': '22 Mar, 2023'}
823         {'coming_soon': True, 'date': '22 Mar, 2023'}
824         {'coming_soon': True, 'date': '22 Mar, 2023'}
                               ...                       
40603        {'coming_soon': True, 'date': 'Coming soon'}
40618        {'coming_soon': True, 'date': 'Coming soon'}
40619        {'coming_soon': True, 'date': 'Coming soon'}
40620        {'coming_soon': True, 'date': 'Coming soon'}
40621        {'coming_soon': True, 'date': 'Coming soon'}
Name: release_date, Length: 170, dtype: object

In [15]:
# Only keep released games
df['coming_soon'] = df['release_date'].apply(lambda x: x['coming_soon'])
df = df[~df['coming_soon']].copy()

# Remove those with blank string 
df['release_date'] = df['release_date'].apply(lambda x: x['date'])
df = df[df['release_date'] != '']

In [16]:
# Format all dates to datetime format
to_remove = ['21 AGO 2012', '9 DIC 2020', '25 AGO 2014']
df = df[~df['release_date'].isin(to_remove)]

df['datetime'] = pd.to_datetime(df['release_date'], format='%d %b, %Y', errors='coerce').fillna(
    pd.to_datetime(df['release_date'], format='%d %b %Y', errors="coerce")).fillna(
    pd.to_datetime(df['release_date'], format='%b %Y', errors="coerce")).fillna(
    pd.to_datetime(df['release_date'], format='%b %d, %Y', errors="coerce")).fillna(
    pd.to_datetime(df['release_date'], format='%Y年%m月%d日', errors="coerce"))

# Does not have a format we can input into to_datetime
rep = {'déc.':'Dec', 'sept.': 'Sep', 'ENE': 'Jan', 'сен.': 'Sep', 'окт.': 'Oct', '. Juli': ' Jul'}
def replace_month(s):
    for key in rep:
        if key in s:
            return s.replace(key, rep[key])
    return s

df['release_date'][df['datetime'].isnull()] = df['release_date'][df['datetime'].isnull()].apply(lambda x: replace_month(x))

df['datetime'] = df['datetime'].fillna(pd.to_datetime(df['release_date'], format='%d %b %Y', errors='coerce'))
print(df['datetime'].isnull().sum())

df['release_date'] = df['datetime']
df.drop(['datetime'], axis=1, inplace=True)
df.head(1)

0


C:\Users\65973\AppData\Local\Temp\ipykernel_21528\2039918382.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'][df['datetime'].isnull()] = df['release_date'][df['datetime'].isnull()].apply(lambda x: replace_month(x))


,type,name,steam_appid,required_age,is_free,developers,publishers,price_overview,platforms,categories,release_date,controller_support,dlc,metacritic,coming_soon
3,dlc,Grand Theft Auto V - Criminal Enterprise Starter Pack,771300,0.0,False,['Rockstar North'],['Rockstar Games'],"{'currency': 'SGD', 'initial': 1390, 'final': 1390, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': 'S$13.90'}","{'windows': True, 'mac': False, 'linux': False}","[Single-player, Multi-player, Downloadable Content, Steam Achievements, Full controller support]",2017-12-14,full,NaN,NaN,False


### Price Overview

In [17]:
df['price_overview'] = df['price_overview'].apply(lambda x: eval(x) if type(x) == str else x)
df['price_overview'].head()

3           {'currency': 'SGD', 'initial': 1390, 'final': 1390, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': 'S$13.90'}
4       {'currency': 'SGD', 'initial': 525, 'final': 393, 'discount_percent': 25, 'initial_formatted': 'S$5.25', 'final_formatted': 'S$3.93'}
9     {'currency': 'SGD', 'initial': 1000, 'final': 670, 'discount_percent': 33, 'initial_formatted': 'S$10.00', 'final_formatted': 'S$6.70'}
10    {'currency': 'SGD', 'initial': 1000, 'final': 670, 'discount_percent': 33, 'initial_formatted': 'S$10.00', 'final_formatted': 'S$6.70'}
11    {'currency': 'SGD', 'initial': 1300, 'final': 871, 'discount_percent': 33, 'initial_formatted': 'S$13.00', 'final_formatted': 'S$8.71'}
Name: price_overview, dtype: object

In [18]:
# Replace NaN with default dicts (impute after train test split)
fix_index = df[df['price_overview'].isnull()].index
lst = []
for value in df['price_overview']:
    if type(value) == dict:
        lst.append(value)
    else:
        lst.append({
            'currency': '', 
            'initial': None, 
            'final': None, 
            'discount_percent':None, 
        })
df['price_overview'] = lst
df[df['price_overview'].isnull()]

,type,name,steam_appid,required_age,is_free,developers,publishers,price_overview,platforms,categories,release_date,controller_support,dlc,metacritic,coming_soon


In [19]:
df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
df['initial'] = df['price_overview'].apply(lambda x: x['initial'])
df['final'] = df['price_overview'].apply(lambda x: x['final'])
df['discount_percent'] = df['price_overview'].apply(lambda x: x['discount_percent'])

In [20]:
# pip install currencyconverter
df[['currency', 'initial']]

,currency,initial
3,SGD,1390.0
4,SGD,525.0
9,SGD,1000.0
10,SGD,1000.0
11,SGD,1300.0
...,...,...
40612,SGD,110.0
40613,,NaN
40614,SGD,325.0
40616,SGD,325.0


In [21]:
# Convert initial and final price to USD using release date for non-empty rows
from currency_converter import CurrencyConverter
c = CurrencyConverter(fallback_on_missing_rate=True, fallback_on_wrong_date=True)
list_currencies = list(c.currencies)

def convert(x, money_var):
    if x['currency'] in list_currencies: # and not isinstance(x['release_date'], str):
        return c.convert(x[money_var], x['currency'], 'USD', x['release_date'])
    else:
        return None

df['initial'] = df.apply(lambda x: convert(x, 'initial'), axis=1)
df['final'] = df.apply(lambda x: convert(x, 'final'), axis=1)

df.head(1)

,type,name,steam_appid,required_age,is_free,developers,publishers,price_overview,platforms,categories,release_date,controller_support,dlc,metacritic,coming_soon,currency,initial,final,discount_percent
3,dlc,Grand Theft Auto V - Criminal Enterprise Starter Pack,771300,0.0,False,['Rockstar North'],['Rockstar Games'],"{'currency': 'SGD', 'initial': 1390, 'final': 1390, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': 'S$13.90'}","{'windows': True, 'mac': False, 'linux': False}","[Single-player, Multi-player, Downloadable Content, Steam Achievements, Full controller support]",2017-12-14,full,NaN,NaN,False,SGD,1032.260188,1032.260188,0.0


In [22]:
df.drop(['coming_soon', 'currency', 'price_overview'], axis=1, inplace=True)
df.head()

,type,name,steam_appid,required_age,is_free,developers,publishers,platforms,categories,release_date,controller_support,dlc,metacritic,initial,final,discount_percent
3,dlc,Grand Theft Auto V - Criminal Enterprise Starter Pack,771300,0.0,False,['Rockstar North'],['Rockstar Games'],"{'windows': True, 'mac': False, 'linux': False}","[Single-player, Multi-player, Downloadable Content, Steam Achievements, Full controller support]",2017-12-14,full,NaN,NaN,1032.260188,1032.260188,0.0
4,dlc,Unturned - Permanent Gold Upgrade,306460,0.0,False,['Smartly Dressed Games'],['Smartly Dressed Games'],"{'windows': True, 'mac': True, 'linux': True}","[Single-player, Multi-player, Co-op, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, Steam Cloud, Stats, Includes level editor]",2014-07-07,NaN,NaN,NaN,421.140227,315.253541,25.0
9,dlc,Rust - Instruments Pack,1174370,17.0,False,['Facepunch Studios'],['Facepunch Studios'],"{'windows': True, 'mac': True, 'linux': False}","[Multi-player, MMO, PvP, Online PvP, Co-op, Online Co-op, Cross-Platform Multiplayer, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, In-App Purchases, Valve Anti-Ch...",2019-12-05,NaN,NaN,NaN,734.556048,492.152552,33.0
10,dlc,Rust - Sunburn Pack,1353060,17.0,False,['Facepunch Studios'],['Facepunch Studios'],"{'windows': True, 'mac': True, 'linux': False}","[Multi-player, MMO, PvP, Online PvP, Co-op, Online Co-op, Cross-Platform Multiplayer, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, In-App Purchases, Valve Anti-Ch...",2020-07-09,NaN,NaN,NaN,719.031317,481.750983,33.0
11,dlc,Rust - Voice Props Pack,1670430,17.0,False,['Facepunch Studios'],['Facepunch Studios'],"{'windows': True, 'mac': True, 'linux': False}","[Multi-player, MMO, PvP, Online PvP, Co-op, Online Co-op, Cross-Platform Multiplayer, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, In-App Purchases, Valve Anti-Ch...",2021-07-01,NaN,NaN,NaN,966.239290,647.380324,33.0


### Platforms

In [23]:
# Since there are 3 fixed platforms, encode before train-test split

def fix_platforms(x):
    windows = 0
    mac = 0
    linux = 0
    if type(x) == str:
        data = eval(x)
        if data['windows'] == True:
            windows = 1
        if data['mac'] == True:
            mac = 1
        if data['linux'] == True:
            linux = 1
    return windows, mac, linux
        
df['platforms'].unique()

array(["{'windows': True, 'mac': False, 'linux': False}",
       "{'windows': True, 'mac': True, 'linux': True}",
       "{'windows': True, 'mac': True, 'linux': False}",
       "{'windows': True, 'mac': False, 'linux': True}",
       "{'windows': False, 'mac': True, 'linux': False}",
       "{'windows': False, 'mac': False, 'linux': True}"], dtype=object)

In [24]:
res = df['platforms'].apply(fix_platforms)

In [25]:
windows = []
mac = []
linux = []
for tup in res:
    windows.append(tup[0])
    mac.append(tup[1])
    linux.append(tup[2])
df['windows'] = windows
df['mac'] = mac
df['linux'] = linux
df = df.drop('platforms', axis = 1)
df.head(1)

,type,name,steam_appid,required_age,is_free,developers,publishers,categories,release_date,controller_support,dlc,metacritic,initial,final,discount_percent,windows,mac,linux
3,dlc,Grand Theft Auto V - Criminal Enterprise Starter Pack,771300,0.0,False,['Rockstar North'],['Rockstar Games'],"[Single-player, Multi-player, Downloadable Content, Steam Achievements, Full controller support]",2017-12-14,full,NaN,NaN,1032.260188,1032.260188,0.0,1,0,0


### Controller Support

In [26]:
df['controller_support'].unique()

array(['full', nan], dtype=object)

In [27]:
# Replace "full" with 1 and nan with 0
df['controller_support'] = df['controller_support'].apply(lambda x: 1 if not pd.isnull(x) else 0)


### DLC

In [28]:
# Replace NaN with empty list
df['dlc'] = df['dlc'].fillna('[]')
df['dlc'].head()

3     []
4     []
9     []
10    []
11    []
Name: dlc, dtype: object

### Metacritics

In [29]:
# Leave NaN as is (impute after train test split)
df['metacritic_score'] = df['metacritic'].apply(lambda x: eval(x)['score'] if not pd.isnull(x) else None)
df['metacritic_url'] = df['metacritic'].apply(lambda x: eval(x)['url'] if not pd.isnull(x) else None)

In [30]:
df.drop(['metacritic'], axis=1, inplace=True)


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36063 entries, 3 to 40617
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   type                36063 non-null  object        
 1   name                36063 non-null  object        
 2   steam_appid         36063 non-null  int64         
 3   required_age        36063 non-null  float64       
 4   is_free             36063 non-null  object        
 5   developers          36063 non-null  object        
 6   publishers          36063 non-null  object        
 7   categories          36063 non-null  object        
 8   release_date        36063 non-null  datetime64[ns]
 9   controller_support  36063 non-null  int64         
 10  dlc                 36063 non-null  object        
 11  initial             32945 non-null  float64       
 12  final               32945 non-null  float64       
 13  discount_percent    32982 non-null  float64   

In [32]:
df.to_csv("../Data/steam_store_dlc_clean.csv")